Importieren grundlegender Packages

In [1]:
# Dataframes und Datenanalyse-Funktionen
import pandas as pd
# Warnung für chained assignments ausschalten
pd.options.mode.chained_assignment = None

# Numerische Berechnungen mit Dataframes/Listen etc. 
import numpy as np

# "Simple" Diagramme
import matplotlib.pyplot as plt

# Komplexe Diagramme (exportierbar in html)
import seaborn as sns

# Einfache Klasse zum herunterladen von Google Drive Dateien
from google_drive_downloader import GoogleDriveDownloader as gdd

# Miscellaneous operating system interfaces
import os as os

# Basic date and time types
import datetime as dt

# Regular expressions
import re

Definition des Chat-Dataframes

In [116]:
# Import der History.txt. Getrennt in 2 Spalten durch das ] Zeichen am Ende des Timestamps
# Spaltennamen als Timestamp und name_and_message definiert. Keine Kopfzeile.
# Text und Name muss noch getrennt werden. 
chat = pd.read_csv('Chatlog.txt', delimiter = ']', names = ['timestamp', 'name_and_message'], header = None)

# Leere Spalte für messages eingefügt
chat['message'] = ''

# Durchläuft den Chat von hinten nach vorne und prüft ob name_and_message ein missing value ist. Wenn ja, befindet sich die Nachricht im Timestamp-Feld und wird in das
# message Feld des vorherigen Eintrags eingetragen und dazu summiert. So Befindet sich die gesamte Nachricht beim zugehörigen Timestamp

for row_number in reversed(range(len(chat))):
    if pd.isna(chat.name_and_message[row_number]):
        chat['message'][row_number-1] =  chat.timestamp[row_number] + chat['message'][row_number]

# Die Zeilen mit missing values wurden so bearbeitet und können entfernt werden.

chat.dropna(inplace = True)
chat.reset_index(inplace = True, drop = True)

# Aufbereiten der timestamp-column zur Umwandlung des Objekts in ein Datetime-Objekt
# Entfernen des [ am Start des Timestamps

chat.timestamp.replace(regex = True, inplace = True, to_replace = r'\u200e', value = r'')

# Erkennen ob in timestamp ein Datum steht, das für einen reply spricht (Jahr und Sekunde fehlt).



chat['replied'] = chat.timestamp.str.contains('\[\d{2}\.\d+\.\, \d+\:\d+')
# Umwandlung zu Datetime

#chat['timestamp'] = pd.to_datetime(chat['timestamp'], format = '[%d.%m.%y, %H:%M:%S')

chat.tail()

,timestamp,name_and_message,message,replied
89995,"[27.04.19, 18:51:49",Tim Ackermann: ‎Bild weggelassen,,False
89996,"[27.04.19, 19:11:07",Yannic Büchele: ?,,False
89997,"[27.04.19, 19:22:46",emptytides: .,,False
89998,"[27.04.19, 19:26:15",Yannic Büchele: Echt so,,False
89999,"[27.04.19, 19:26:18",Yannic Büchele: Richtig was los hier,,False


In [117]:
chat['replied'] = chat.timestamp.str.contains('\[\d{2}\.\d+\.\, \d+\:\d+|\[\d{2}\:\d{2}\, \d+\/\d+\/\d{4}' )

# Anmerkung: Erkennt alle Timestamps als True ... WHY?!

for row_number in reversed(range(len(chat))):
    if chat['replied'][row_number] == True:
        chat['message'][row_number-1] = 'reply:' + chat.name_and_message[row_number]
        chat['replied'][row_number-1] = True

chat = chat[chat.timestamp.str.contains('\[\d{2}\.\d{2}\.\d{2}, \d{2}\:\d{2}\:\d{2}')]

chat['timestamp'] = pd.to_datetime(chat['timestamp'], format = '[%d.%m.%y, %H:%M:%S')

In [21]:
whatsapp_file = 'Chatlog.txt'

# Read Whatsapp File as single string
if os.path.exists(whatsapp_file):
    print('True')
    file_data = open(whatsapp_file, 'r', encoding = "utf8", errors = 'ignore')
    whatsapp_content = file_data.read()
    
# Get all Dates from the string
date_regex = re.compile(r'(\d+/\d+/\d+)')
date = date_regex.findall(whatsapp_content)

# Get all times from the string
time_regex = re.compile(r'(24:00|2[0-3]:[0-5][0-9]|[0-1][0-9]:[0-5][0-9])')
time = time_regex.findall(whatsapp_content)

# Get the Users from the string
user_regex = re.compile(r'-(.*?):')
user = user_regex.findall(whatsapp_content)

# Get the message text from the string
message_regex = re.compile(r"(\n)(?<=)(\d+/\d+/\d+)(.*)")
message = message_regex.findall(whatsapp_content)

# Zip the data into one list of lists
data = []
for w,x,y,z in zip(date, time, user, message):
    data.append([str(w),str(x),str(y),str(z)])

# Transform into a pandas DataFrame
chat_df = pd.DataFrame(data, columns = ('Date', 'Time', 'User', 'Message'))

# remove special characters from message
chat_df['message'] = chat_df['Message'].str.replace('\'(.*?): ','')

# Change date-string into datetime
chat_df['Date'] = pd.to_datetime(chat_df['Date'])

True


In [28]:
user

['Wrttemberg 72072)',
 '8728?',
 'Paradoxons ist folgende berlegung',
 'Dossier',
 ' Hickory, NC (1179  13th Ave Dr SE, Hickory, NC, 28602)',
 'Wrttemberg 76137)',
 '5, Hamburg, Hamburg 20099)',
 '0-16 mr15 cs',
 'Lounge der Commerzbank-Arena (Mrfelder Lanstrae 362, Frankfurt, 60528)',
 '7, Saarbrcken, 66111)',
 'Josef-Strau -Allee, Regensburg)',
 'Allee 10-20, Cologne)',
 '19',
 'Allee 10-20, Cologne)',
 ' und Universittsbibliothek Carl von Ossietzky (Von-Melle-Park 3, Hamburg, Hamburg 20146)',
 ' dachte we und ich wre individuell ',
 'Schauernheim, 67125)',
 'Rhein-Ufer 1, Hamburg, Hamburg 22087)',
 'Humbert-Str. 151, Bremen, 28759)',
 'Wrttemberg 76131)',
 'Wrttemberg 76131)',
 'Mayer-Str. 1, Regensburg)',
 '15, Mundsburg Center, Hamburg, Hamburg 22083)',
 ' Juniorfirma (Mies-van-der-Rohe-Str. 6, Munich)',
 ' - - Buchempfehlungen',
 ' und Outdoorsport an (Kletterkurse, Trainingsgruppen, Kletterreisen, Workshops,). queerclimb mchte alle Altersklassen sowie Menschen unterschiedlichste